# Asyncio Basics

## The non-asyncio approach using threads

In [1]:
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor

In [2]:
from contextlib import contextmanager

In [3]:
def slow(x):
    print(x)
    time.sleep(0.05)
    return x

def main_synchronous():
    res = []
    for i in range(10):
        res.append(slow(i))
    return res

def main_threads():
    with ThreadPoolExecutor(10) as pool:
        res = list(pool.map(slow, range(10)))
    return res

In [4]:
%timeit main_threads()

0
12

3
45

6
78

9
01

23

4
56

7
89

01

23

4
5
6
7
89

0
1
23

4
56

7
89

01

23

4
56

7
89

01

2
3
4
5
6
7
8
9
01

2
3
45

6
78

9
01

2
3
4
5
6
7
8
9
01

23

4
56

7
89

01

2
34

5
6
7
8
9
0
12

3
45

6
78

9
0
1
2
3
4
5
67

8
9
01

2
3
45

6
78

9
01

2
34

5
6
7
8
9
01

2
34

5
6
7
8
9
01

2
3
45

6
78

9
01

23

4
56

7
89

0
12

3
45

6
7
8
9
01

2
3
4
5
67

8
9
0
12

3
4
56

7
89

01

23

4
56

7
89

01

2
3
4
56

7
89

01

2
34

5
67

8
9
01

2
3
45

6
78

9
01

2
3
45

6
78

9
01

2
34

5
67

8
9
0
12

3
4
5
67

8
9
01

2
3
4
5
67

8
9
01

23

4
56

7
89

01

2
3
4
5
67

8
9
0
1
23

4
56

7
89

01

2
3
4
5
67

8
9
0
1
23

4
56

7
89

01

2
3
4
56

7
89

01

2
3
4
5
6
78

9
0
1
2
3
45

6
78

9
01

23

4
5
67

8
9
0
1
2
3
4
5
67

8
9
01

2
3
4
5
6
7
8
9
01

2
3
4
5
6
7
8
9
01

2
34

5
67

89

01

2
3
4
5
6
78

9
01

2
3
45

67

8
9
01

2
3
4
5
6
7
8
9
01

2
34

5
67

8
9
01

23

4
56

7
8
9
01

2
3
4
5
67

8
9
0
1
2
3
4
5
6
7
8
9
01

23

4
5
6
78

9
01

2
3
4
5
6
7
8
9


## The asyncio approach
Rewriting *everything* using async/await

### No limit on concurrency
Use `asyncio.wait` to fire all coroutines. 

In [5]:
async def slow_async(x):
    print(x)
    import random
    s = random.random()
    await asyncio.sleep(s)
    return x

async def main_async_nolimit():
    """Fire all tasks at once"""
    t0 = time.perf_counter()
    done, pending = await asyncio.wait([slow_async(i) for i in range(10)], timeout=5)
    res = [task.result() for task in done]
    t1 = time.perf_counter()
    print(t1 - t0)
    return res

In [6]:
await main_async_nolimit()

5
6
7
3
8
0
2
9
1
4
0.8237295527942479


[7, 3, 9, 6, 8, 5, 1, 0, 4, 2]

### Controlled concurrency
Limit the number of concurrent tasks via the `return_when='FIRST_COMPLETED'` parameter of `asyncio.wait`

In [7]:
async def main_async_pool1(max_workers=None, N=10):
    """Limits the number of concurerent tasks"""
    
    if max_workers is None:
        max_workers = N
    t0 = time.perf_counter()
    tasks = [slow_async(i) for i in range(10)]
    pending = set()
    all_done = set()
    while tasks:
        #print('len(tasks) = {}, len(queue) = {}'.format(len(tasks), len(queue)))
        while True:
            if tasks and len(pending) < max_workers:
                #print('pushing 1 tasks')
                pending.add(tasks.pop())
            else:
                break
        if pending:
            done, pending = await asyncio.wait(pending, return_when='FIRST_COMPLETED')  
            #return {'done': done, 'pending': pending}
            all_done.update(done)
    res = [task.result() for task in all_done]
    t1 = time.perf_counter()
    print(t1 - t0)
    return res

await main_async_pool1(3)

7
9
8
6
5
4
3
2
1
0
1.3626035328488797


[8, 7, 5, 4, 9, 6, 3, 1]

**better version using contextmanager**

This sets up a async Pool executor via a context manager similar to what's available in the concurrent.futures package, so you can map a function over an iterable asynchronously. 


In [8]:
class AsyncPoolExecutor:
    """Setting up an async Pool of executors with asyncio with an API similar to concurrent.futures"""
    
    def __init__(self, max_workers=None):
        self.max_workers = max_workers
        
    def __enter__(self):
        return self 
    
    def __exit__(self, *args, **kwargs):
        pass 
    
    async def map(self, foo, iterable):
        pending = set()
        finished_tasks = set()
        tasks = [foo(i) for i in iterable]
        while tasks or pending:
            #print('len(tasks) = {}, len(pending) = {}'.format(len(tasks), len(pending)))
            while True:
                if tasks and len(pending) < self.max_workers:
                    #print('pushing 1 tasks')
                    pending.add(tasks.pop())
                else:
                    break
            done, pending = await asyncio.wait(pending, return_when='FIRST_COMPLETED')  
            #return {'done': done, 'pending': pending}
            finished_tasks.update(done)
        #print('finished_tasks = ', finished_tasks)
        return [task.result() for task in finished_tasks]
    
    

async def main_async_pool2(max_workers=None, N=10):
    """Fire all tasks at once"""
    
    if max_workers is None:
        max_workers = N
    t0 = time.perf_counter()
    res = []
    with AsyncPoolExecutor(max_workers=max_workers) as pool:
        res = await pool.map(slow_async, range(10))
    t1 = time.perf_counter()
    print(t1 - t0)
    return res

In [9]:
await main_async_pool2()

3
7
4
0
8
1
9
2
6
5
0.8068548848386854


[2, 1, 8, 9, 5, 7, 0, 4, 6, 3]

In [10]:
loop = asyncio.get_event_loop()

## Using AsyncIO with blocking code
Sometimes you don't control the code you'd like to use with asyncio. For example boto3 is synchronous, and you may not want to use some of the functionality without having to rewrite the whole library. For such cases, you can combine `asyncio` with an PoolExecutor from `concurrent.futures` to turn a blocking function into an ansynchronous one that can be awaited and is compatible with asyncio. 

The trick is to`loop.run_in_executor`.

In [291]:
async def main_async_run_in_executor(max_workers=None, N=10):
    """This uses a pool of executors and a blocking function! Nice to wrap existing code"""
    t0 = time.perf_counter()
    res = []
    with ThreadPoolExecutor(max_workers=max_workers) as pool:
        loop = asyncio.get_event_loop()
        # loop.run_in_executor runs blocking code is separate threads/processes to make it async
        # note that ThreadPoolExecutor(max_workers=None) is actually the default, so we could set `pool` below 
        # to None and get that behaviour. 
        tasks = [loop.run_in_executor(pool, slow, i) for i in range(N)]
        done, pending = await asyncio.wait(tasks)
        res = [task.result() for task in done]
    t1 = time.perf_counter()
    print(t1 - t0)
    return res

In [12]:
await main_async_run_in_executor()

0
1
2
3
4
5
6
78

9
0.05668182508088648


[8, 9, 6, 1, 7, 3, 4, 0, 2, 5]

## Tasks vs Futures vs Coroutines
### Coroutine as generators
Like a blank space representing a value that will be populated asynchronously at some point in the future. That blank space is the return value of the *coroutine function*. Somewhere in the definition of a *coroutine function*, there will be a point where the execution is suspended asynchronously while we wait for the completion of another coroutine. 

This mechanism is implemented via a the `yield from <some_generator>` operator, which suspends the execution until `<some_generator>` returns a value. 



In [179]:
def i_am_a_coroutine():
    """Must yield form another coroutine.
    
    Just using asyncio.sleep for now!
    """
    yield  
    return 'Coroutine says hi!'

In [242]:
def coroutine_as_generator(coroutine_function): 
    """Turn coroutine into generator: yield None while result isn't ready yet, else yield the result"""
    coroutine = coroutine_function()
    future = asyncio.ensure_future(coroutine)
    while True:
        if not future.done():
            time.sleep(0.1)
            # yielding nothing if future isn't completed
            yield None
        else:
            # otherwise yielding the result
            yield future.result()
            break

In [252]:
g = coroutine_as_generator(i_am_a_coroutine)

In [264]:
try:
    res = next(g)  # keep executing this by hand until you get a result (or Stop Iteration)
    if res is None:
        print('Result not ready yet. Please try again.')
except StopIteration:
    pass
if res is not None:
    print(res)

Coroutine says hi!


### Asyncio Coroutines
Coroutines that are compatible with asyncio must yield from asynchronous objects like futures, tasks or other coroutines. 
Let's see how this works using the `asyncio.sleep` coroutine, which sleeps for 2 seconds asynchronously. 

Here we still use the old `yield from` like before to await the result from `asyncio.sleep`. We can start by using the same "coroutine_as_generator" function to run our coroutine.
This demonstrates that there's no need for any additional magic to run asyncio coroutines. They're basically just generators. 

In [269]:
def coroutine():
    print('Waiting 2 seconds')
    yield from asyncio.sleep(2)
    return 1

In [270]:
g = coroutine_as_generator(coro)

In [273]:
try:
    res = next(g)  # keep executing this by hand until you get a result (or Stop Iteration)
    if res is None:
        print('Result not ready yet. Please try again.')
except StopIteration:
    pass
if res is not None:
    print(res)

1


### Awaiting coroutines
If we use the `asyncio.coroutine` decorator we can now await our coroutine. This automates the repeated calls to `next()` while we wait for the coroutine to complete its execution. 

In [280]:
awaitable_coroutine_function = asyncio.coroutine(coroutine)
await awaitable_coroutine_function()

Waiting 2 seconds


1

So we could have written our coroutine as 

In [283]:
@asyncio.coroutine
def coroutine_func():
    print('Waiting 2 seconds')
    yield from asyncio.sleep(2)
    return 1

await coroutine_func()

Waiting 2 seconds


1

### Async sugar
Using the `@asyncio.coroutine` decorator is the old style way of defining coroutines. The new way is to use the `async def` keyword to define the coroutine function. The only difference with the previous example is we can no longer use `yield from`: `yield form` must be replaced by `await`. So we now have

In [286]:
async def coroutine_func():
    print('Waiting 2 seconds')
    await asyncio.sleep(2)
    return 1

await coroutine_func()

Waiting 2 seconds


1

### Coroutine vs Coroutine Function

Confusingly, calling a coroutine (function) defined via `async_def` & `await` (or `asyncio.coroutine` and `yield from`) returns a coroutine. This is similar to generators, where the term generator is used both to denote the `generator function`, and the output of the generator function. 

In [288]:
coroutine_func()

<coroutine object async-def-wrapper.<locals>.coroutine_func at 0x116197d58>

[From Python 3.6 documentation on coroutines](https://docs.python.org/3.6/library/asyncio-task.html?highlight=coroutine#coroutines)
> The word “coroutine”, like the word “generator”, is used for two different (though related) concepts:
>  - The function that defines a coroutine (a function definition using async def or decorated with @asyncio.coroutine). If disambiguation is needed we will call this a coroutine function (iscoroutinefunction() returns True).
>  - The object obtained by calling a coroutine function. This object represents a computation or an I/O operation (usually a combination) that will complete eventually. If disambiguation is needed we will call it a coroutine object (iscoroutine() returns True).


### Future

An object "encapsulating the asynchronous execution of a callable". The `asyncio.Future` class is very similar to the `concurrent.futures.Future` class. Makes it possible to represent an asynchronous execution and to pass it around. It's largely the same as a coroutine (not a *coroutine function*). The main difference is that it has a few more methods allowing to control the execution (if it's not completed yet):
- futures can be cancelled by calling the `cancel` method
- their status can be monitored via the `cancelled`, `running` and `done` methods

In [289]:
async def my_name_is(future):
    await asyncio.sleep(1)
    print(f'(Is future {future} cancelled? {future.cancelled()})')
    print(f'(Is future {future} done? {future.done()})')
    future.set_result('Sam')

async def whats_your_name():
    print("What's your name mate?")
    future = asyncio.Future()
    coroutine = my_name_is(future)
    await coroutine
    print(f'(Is future {future} done? {future.done()})')
    print(f"My name is {future.result()}")
    


In [290]:
await whats_your_name()

What's your name mate?
(Is future <Future pending> cancelled? False)
(Is future <Future pending> done? False)
(Is future <Future finished result='Sam'> done? True)
My name is Sam


## References

- Nice and simple 15mn intro (views): https://www.youtube.com/watch?v=tSLDcRkgTsY
- Another short intro (30mn, 71k views): https://www.youtube.com/watch?v=BI0asZuqFXM